<a href="https://colab.research.google.com/github/mini1115/TIL/blob/master/%EB%B9%85%EB%B6%84%EA%B8%B0%20%EA%B8%B0%EC%B6%9C/6%ED%9A%8C%EC%B0%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()

Saving energy_train.csv to energy_train.csv
Saving energy_test.csv to energy_test.csv
Saving data6-3-2.csv to data6-3-2.csv
Saving data6-1-3.csv to data6-1-3.csv
Saving data6-1-2.csv to data6-1-2.csv
Saving data6-1-1.csv to data6-1-1.csv


# **작업형1**

Q1. 출동시간과 도착시간 차이가 평균적으로 가장 오래 걸린 소방서의 시간을 분으로 변환하여 출력하시오.

In [3]:
import pandas as pd
df = pd.read_csv('data6-1-1.csv')
df.head()

,소방서,출동시간,도착시간
0,소방서1,2023-09-16 06:12:29,2023-09-16 07:51:55
1,소방서5,2023-09-19 06:11:41,2023-09-19 08:08:36
2,소방서1,2023-09-14 06:12:00,2023-09-14 07:24:21
3,소방서6,2023-09-20 06:12:24,2023-09-20 06:30:45
4,소방서9,2023-09-15 06:11:50,2023-09-15 06:44:00


In [30]:
import datetime
df['출동시간']=pd.to_datetime(df['출동시간'])
df['도착시간']=pd.to_datetime(df['도착시간'])

df['diff'] = df['도착시간']-df['출동시간']
ans = df.groupby('소방서')['diff'].mean()
result = ans.dt.seconds/60
round(result.max())

81

Q2. 학교에서 교사 한 명당 맡은 학생 수가 가장 많은 학교를 찾고, 그 학교의 교사 수를 구하시오.

In [43]:
df = pd.read_csv('data6-1-2.csv')
df['total'] = (df['1학년']+df['2학년']+df['3학년']+df['4학년']+df['5학년']+df['6학년']) / df['교사수']
df = df.sort_values('total',ascending=False)
df.iloc[0]['교사수']

19

Q3. 연도별로 총 범죄 건수(범죄 유형별 발생 건수의 합)의 월평균 값을 구한 후<BR/>그 값이 가장 큰 연도를 찾아 해당 연도의 총 범죄 건수의 월평균 값을 출력하시오(반올림하여 정수 출력)

In [80]:
df = pd.read_csv('data6-1-3.csv')

import datetime
df['year'] = df['날짜'].str.split(" ").str[0]
df['month'] = df['날짜'].str.split(" ").str[1]
df['total'] = df.iloc[:,1:7].sum(axis=1)

ans = df['total'].groupby(df['year']).sum()
result = round(ans.max()/12)
result

533

# **작업형2**

Q. 건물 데이터를 기반으로 건물의 난방 부하(에너지 효율성)를 예측하시오.
  - 제공된 컬럼 : train, test
  - 예측할 컬럼 : Heat_Load(Very Low,Low,Medium,High,Very High)

  - 평가지표 : macro F1-score


  

In [114]:
train = pd.read_csv('energy_train.csv')
test = pd.read_csv('energy_test.csv')

# train.shape,test.shape
# ((537, 10), (231, 9))
# train.head()
# train.info()
# train.describe(include='O')
# train.isnull().sum()
# train['Heat_Load'].value_counts()
target = train.pop('Heat_Load')

# combined = pd.concat([train,test])
# combined_oh = pd.get_dummies(combined)
# train = combined_oh[:len(train)]
# test = combined_oh[len(train):]

from sklearn.preprocessing import LabelEncoder
cols = train.select_dtypes(include='O').columns
for col in cols:
  le = LabelEncoder()
  train[col] = le.fit_transform(train[col])
  test[col] = le.transform(test[col])


from sklearn.model_selection import train_test_split
X_tr,X_val,y_tr,y_val = train_test_split(train,target,test_size=0.2,random_state=0)
# X_tr.shape,X_val.shape,y_tr.shape,y_val.shape

# from sklearn.ensemble import RandomForestClassifier
# rf = RandomForestClassifier(max_depth=9,random_state=0)
# rf.fit(X_tr,y_tr)
# pred = rf.predict(X_val)
# 0.9290

import lightgbm as lgb
lg = lgb.LGBMClassifier(max_depth=9,random_state=0,verbose=-1)
lg.fit(X_tr,y_tr)
pred = lg.predict(X_val)
# 0.9430

from sklearn.metrics import f1_score
score = f1_score(y_val,pred,average='macro')
print(score)

pred = lg.predict(test)
submit = pd.DataFrame({'pred':pred})
submit.to_csv('result.csv',index=False)
pd.read_csv('result.csv').head(5)

0.9430631314043516


,pred
0,Low
1,High
2,High
3,Low
4,Low


# **작업형 3유형**
Q. 감기약 위약(플리시보)을 투여받은 환자의 부작용은 항암약 위약(플리시보)을 투여받은 환자의 부작용과 차이가 있는가?

In [115]:
import pandas as pd
df = pd.DataFrame({
    "항암약":[4,4,3,4,1,4,1,4,1,4,4,2,1,4,2,3,2,4,4,4]
})
print(df.head())

   항암약
0    4
1    4
2    3
3    4
4    1


- 귀무가설 : 감기약의 부작용과 항암약의 부작용은 동일하다.
- 대립가설 : 감기약의 부작용과 항암약의 부작용은 다르다.

Q1. 항암약을 투여받은 환자 중 '이상없음' 비율을 0과 1사이로 나타내시오.
(1: 아픔 , 2 : 조금 아픔, 3 : 속쓰림, 4 : 이상없음)

In [119]:
ans = df.loc[df['항암약']==4]
ratio = len(ans) / len(df)
ratio

0.55

Q2. 감기약 예상 부작용 비율과 항암약의 부작용 관찰값이 통계적으로 유의미하게 차이가 있는지 확인하려 한다. 카이제곱 검정 통계략을 구하시오.

In [134]:
from scipy.stats import chisquare

# df['항암약'].value_counts() / len(df)

ratio = df['항암약'].value_counts().sort_index().to_list()
side_effect = [0.1,0.05,0.15,0.7]
for x in side_effect:
  expected = len(df) * x
  print(expected)

expected= [2,1,3,14]

help(chisquare)
# chisquare(f_obs, f_exp=None, ddof=0, axis=0)
chisquare(ratio,expected)

2.0
1.0
3.0
14.0
Help on function chisquare in module scipy.stats._stats_py:

chisquare(f_obs, f_exp=None, ddof=0, axis=0)
    Calculate a one-way chi-square test.
    
    The chi-square test tests the null hypothesis that the categorical data
    has the given frequencies.
    
    Parameters
    ----------
    f_obs : array_like
        Observed frequencies in each category.
    f_exp : array_like, optional
        Expected frequencies in each category.  By default the categories are
        assumed to be equally likely.
    ddof : int, optional
        "Delta degrees of freedom": adjustment to the degrees of freedom
        for the p-value.  The p-value is computed using a chi-squared
        distribution with ``k - 1 - ddof`` degrees of freedom, where `k`
        is the number of observed frequencies.  The default value of `ddof`
        is 0.
    axis : int or None, optional
        The axis of the broadcast result of `f_obs` and `f_exp` along which to
        apply the test.  If

Power_divergenceResult(statistic=6.976190476190476, pvalue=0.07266054733847573)

Q2. 다중 선형 회귀 모델을 구축하고 각 소문제에 답하시오.
- 독립변수 : SOLAR, WIND, O3
- 종속변수 : TEMPERATURE

In [135]:
df = pd.read_csv('data6-3-2.csv')
df.head()

,solar,wind,o3,temperature
0,89.14,6.28,33.52,23.0
1,109.97,1.04,27.01,20.7
2,102.83,6.42,41.00,20.5
3,84.94,10.20,33.44,22.2
4,94.21,4.95,29.97,21.4


Q2-1. 주어진 데이터를 기반으로 다중 선형 회귀 모델을 구축하고 , 독립변수 O3의 회귀계수를 구하시오.

0.0749

In [137]:
from statsmodels.formula.api import ols

formula = 'temperature ~ solar + wind + o3'
model = ols(formula,data=df).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            temperature   R-squared:                       0.044
Model:                            OLS   Adj. R-squared:                  0.014
Method:                 Least Squares   F-statistic:                     1.464
Date:                Mon, 25 Nov 2024   Prob (F-statistic):              0.229
Time:                        05:29:23   Log-Likelihood:                -195.45
No. Observations:                 100   AIC:                             398.9
Df Residuals:                      96   BIC:                             409.3
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     19.0507      1.994      9.555      0.000      15.093      23.008
solar          0.0039      0.015      0.251      0.802      -0.027       0.035
wind          -0.0252      0.090     -0.280      0.780      -0.204       0.153
o3             0.0749      0.036      2.079      0.040       0.003       0.146
==============================================================================
Omnibus:                        0.654   Durbin-Watson:                   2.328
Prob(Omnibus):                  0.721   Jarque-Bera (JB):                0.672
Skew:                           0.187   Prob(JB):                        0.715
Kurtosis:                       2.855   Cond. No.                     1.20e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.2e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Q2-2. 문제 2-1에서 적합한 모델에서 SOLAR와 O3이 고정된 상태에서 <BR/>
WIND의 세기가 증가함에 따라 TEMPERATUR가 감소한다
이 회귀모델에서 'WIND'의 회귀계수에 대한 P_VALUE를 구하시오

In [139]:
model.pvalues['wind']

0.7797177202071661

Q2-3. 문제 2-1에서 적합한 모델에서 SOLAR : 100, WIND : 5, O3 : 30
일때 예측값을 구하시오.

In [144]:
data = pd.DataFrame({'solar':[100],'wind':[5],'o3':[30]})
new = model.predict(data)
new

,0
0,21.56163
